In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import anndata

In [2]:
#select replicate indices and save their sum
def removeDots(adata):
    #find genes with a '.' in their name
    noreplica = adata.var.loc[~adata.var.index.str.contains("\\."),:].index
    replicas = adata.var.loc[adata.var.index.str.contains("\\."),:].index
    uniques = set()
    new_var_names = set()
    for x in replicas: 
        uniques.add(x.split(".")[0])
        new_var_names.add(x.split(".")[0])
    new_var_names.update(noreplica.tolist())
    adata_new = sc.AnnData(np.zeros((len(adata.obs_names),len(new_var_names))))
    adata_new.var_names = list(new_var_names)
    adata_new.obs_names = adata.obs_names
    adata_new[:,noreplica].X = adata[:,noreplica].X
    adata_new.obs = adata.obs
    for x in uniques:
        ind = adata.var.loc[adata.var.index.str.startswith(x+"\\."),:].index
        if(len(ind)!=1):
            adata_new[:,x].X = np.sum(adata[:,ind].X,axis=1)
        else:
            adata_new[:,x].X =  adata[:,ind[0]].X
    return adata_new

In [3]:
import scipy as sci
clusters = ['B-cells', 'CD4+ T-cells', 'CD8+ T-cells', 'NK cells', 'Monocytes', 'Neutrophils']
real_prop = pd.read_csv('./write/results/bulk_bulk_pbmc/proportions_orig.csv',delimiter=',',index_col=0)


In [4]:
real_prop = real_prop.drop('Progenitors',axis=1)

In [5]:
sorted_bulks_paper = pd.read_csv('./pbmc/cell19/rnaseq/wholeGenes/ABIS_RNASeq_full.txt', index_col=0, delimiter = '\t')
sorted_bulks_paper.shape

(17487, 17)

In [6]:
#sorted_bulks_cibersort = pd.read_csv('./sorted_bulks_cibersort.txt', index_col=0, delimiter = '\t')
samples = pd.read_csv('./samples_cibersort.txt', index_col=0, delimiter = '\t')

In [7]:
def normalize_proportions(data,copy):
    if copy==True:
        data_copy = data.copy()
    else:
        data_copy = data
    data_copy[data_copy < 0] = 0
    #print(data_copy)
    for raw in data_copy.index:
        sum = data_copy.loc[raw].sum()
        data_copy.loc[raw] = np.divide(data_copy.loc[raw],sum)
    return data_copy

## LM22

In [9]:
LM22 = pd.read_csv('./LM22.txt',index_col=0,delimiter='\t')
LM22

,B cells naive,B cells memory,Plasma cells,T cells CD8,T cells CD4 naive,T cells CD4 memory resting,T cells CD4 memory activated,T cells follicular helper,T cells regulatory (Tregs),T cells gamma delta,...,Monocytes,Macrophages M0,Macrophages M1,Macrophages M2,Dendritic cells resting,Dendritic cells activated,Mast cells resting,Mast cells activated,Eosinophils,Neutrophils
Gene symbol,,,,,,,,,,,,,,,,,,,,,
ABCB4,555.713449,10.744235,7.225819,4.311280,4.605860,7.406442,8.043976,6.469993,7.833082,9.312295,...,6.202496,7.951360,27.557710,121.432277,114.636747,4.872184,4.727985,34.113659,15.030530,14.906888
ABCB9,15.603544,22.094787,653.392328,24.223723,35.671507,30.048192,38.455423,17.604795,46.073657,19.715722,...,20.309426,56.251887,29.380878,40.390260,123.993305,23.384603,53.466757,23.615746,29.786442,33.679147
ACAP1,215.305951,321.621021,38.616872,1055.613378,1790.097170,922.152747,340.883424,1107.798265,1995.482601,280.075719,...,190.879024,48.972217,30.974254,23.055150,61.386550,26.412923,104.200198,73.607932,696.442802,596.025961
ACHE,15.117949,16.648847,22.123737,13.428288,27.187732,18.444927,13.441268,14.805542,24.652712,33.658448,...,16.375268,13.692963,1373.069620,48.585761,16.715474,193.159977,13.948778,20.501249,40.414927,22.766494
ACP5,605.897384,1935.201479,1120.104684,306.312519,744.656599,557.819820,248.546932,711.949744,958.916018,493.969120,...,512.931382,13644.172300,1062.475909,7633.960182,13825.053630,3989.865169,263.497930,323.381277,860.563374,307.142798
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF204P,77.876897,42.946213,41.075062,151.959734,400.663184,168.895130,40.090641,52.228974,83.763177,129.598647,...,29.909746,26.843634,33.021233,27.461275,25.553860,36.165940,38.887463,25.206700,65.487837,27.485546
ZNF222,9.800664,7.203218,11.691368,71.911257,56.642511,15.081330,15.589903,182.938925,10.027797,13.843262,...,19.860610,70.824860,47.029368,148.622218,86.418190,39.918349,222.573611,211.493742,346.556451,26.825597
ZNF286A,737.859904,331.908761,73.882022,218.227036,5.598819,169.313128,190.874843,118.138051,136.868599,246.503817,...,11.043937,13.505845,36.038732,21.875639,15.080863,280.769931,77.840396,25.833293,7.473171,441.313469


In [10]:
sorted_LM22 = sorted_bulks_paper.loc[np.intersect1d(LM22.index,sorted_bulks_paper.index),:]

In [11]:
sorted_LM22.shape

(467, 17)

In [12]:
samples_LM22 = samples.loc[np.intersect1d(samples.index,sorted_LM22.index),:]
samples_LM22.shape

(467, 13)

In [13]:
import datetime
time0 = datetime.datetime.now()
from sklearn.svm import NuSVR
proportions_NuSVR_LM22 = pd.DataFrame(columns=sorted_LM22.columns)
regr_NuSVR = NuSVR(nu=0.5,C=0.5,kernel='linear') 
for column in samples_LM22.columns:
    regr_NuSVR.fit(sorted_LM22, samples_LM22.loc[:,column])
    proportions_NuSVR_LM22.loc[column] = regr_NuSVR.coef_[0]
    
    proportions_NuSVR_LM22.index = [x[:4] for x in proportions_NuSVR_LM22.index]
    proportions_NuSVR_LM22 = normalize_proportions(proportions_NuSVR_LM22, copy = False)
time1 = datetime.datetime.now()
print(time1-time0)

0:49:51.583206


In [28]:
proportions_NuSVR_LM22 = proportions_NuSVR_LM22.loc[real_prop.index,real_prop.columns]

In [32]:
clusters = real_prop.columns
for cluster in clusters:
    s,i,r,p,e = sci.stats.linregress(real_prop.loc[:,cluster],proportions_NuSVR_LM22.loc[:,cluster]) 
    print(cluster,r,p)

B Naive 0.7312782554476999 0.006881489291999298
Plasmablasts 0.8740568151591389 0.00020123994648685311
T CD4 Naive 0.4811795334759722 0.11325370417953107
T CD8 Naive 0.1608716206487732 0.6174515601665771
T gd Vd2 0.25030482672209003 0.4326567408215052
T gd non-Vd2 0.8451122751054588 0.0005380223861313786
MAIT 0.580774037717924 0.04768752495197947
NK 0.6323705207533444 0.027350043207945218
pDCs 0.4681393287473925 0.12481321228778851
mDCs 0.6553375480970687 0.020703529493389466
Monocytes C 0.7195060830500573 0.008342319972427572
Neutrophils LD 0.7868655571949719 0.002391542143041787
Basophils LD 0.9637211279226455 4.656686477233481e-07
B Memory 0.6604630054859584 0.01939825540885409
T CD4 Memory 0.24506746264564103 0.44266200195821714
T CD8 Memory 0.8559596243958827 0.0003814680095188462
Monocytes NC+I 0.5206461733273499 0.08265085661401032


## xCell

In [10]:
xcell = pd.read_csv('./xcell_genes.txt',index_col=1,delimiter=';')

In [11]:
sorted_xcell = sorted_bulks_paper.loc[np.intersect1d(xcell.index,sorted_bulks_paper.index),:]

In [12]:
sorted_xcell.shape

(8552, 17)

In [13]:
samples_xcell = samples.loc[np.intersect1d(samples.index,sorted_xcell.index),:]

In [14]:
samples_xcell.shape

(8552, 13)

In [ ]:
import datetime
from sklearn.svm import NuSVR
time0 = datetime.datetime.now()
proportions_NuSVR_xcell = pd.DataFrame(columns=sorted_xcell.columns)
regr_NuSVR = NuSVR(nu=0.5,C=0.5,kernel='linear') 
for column in samples_xcell.columns:
    print(column,datetime.datetime.now())
    regr_NuSVR.fit(sorted_xcell, samples_xcell.loc[:,column])
    proportions_NuSVR_xcell.loc[column] = regr_NuSVR.coef_[0]
    
    proportions_NuSVR_xcell.index = [x[:4] for x in proportions_NuSVR_xcell.index]
    proportions_NuSVR_xcell = normalize_proportions(proportions_NuSVR_xcell, copy = False)
print(time1-time0)

CYFZ_PBMC 2020-07-01 11:31:34.932802


In [ ]:
proportions_NuSVR_xcell.to_csv('./proportions_NuSVR_xcell.txt')

In [4]:
pd.read_csv('./proportions_NuSVR_xcell.txt')

FileNotFoundError: [Errno 2] File ./proportions_NuSVR_xcell.txt does not exist: './proportions_NuSVR_xcell.txt'

In [ ]:
proportions_NuSVR_xcell

In [ ]:
proportions_NuSVR_xcell = proportions_NuSVR_xcell.loc[real_prop.index,real_prop.columns]

In [ ]:
for cluster in clusters:
    s,i,r,p,e = sci.stats.linregress(real_prop.loc[:,cluster],proportions_NuSVR_xcell.loc[:,cluster]) 
    print(cluster,r,p)

## Cibersort

In [8]:
cibersort = pd.read_csv('./cibersortx_signatures_sorted_bulks.txt',index_col=0,delimiter='\t')
cibersort.shape

(5486, 28)

In [9]:
sorted_cibersort = sorted_bulks_paper.loc[np.intersect1d(cibersort.index,sorted_bulks_paper.index),:]

In [10]:
sorted_cibersort.shape

(3214, 17)

In [11]:
samples_cibersort = samples.loc[np.intersect1d(samples.index,sorted_cibersort.index),:]
samples_cibersort.shape

(3214, 13)

In [12]:
import datetime
time0 = datetime.datetime.now()
from sklearn.svm import NuSVR
proportions_NuSVR_cibersort = pd.DataFrame(columns=sorted_cibersort.columns)
regr_NuSVR = NuSVR(nu=0.5,C=0.5,kernel='linear') 
for column in samples_cibersort.columns:
    print(column,time0)
    regr_NuSVR.fit(sorted_cibersort, samples_cibersort.loc[:,column])
    proportions_NuSVR_cibersort.loc[column] = regr_NuSVR.coef_[0]
    
    proportions_NuSVR_cibersort.index = [x[:4] for x in proportions_NuSVR_cibersort.index]
    proportions_NuSVR_cibersort = normalize_proportions(proportions_NuSVR_cibersort, copy = False)
time1 = datetime.datetime.now()
print(time1-time0)

CYFZ_PBMC 2020-05-18 14:09:40.159285
FY2H_PBMC 2020-05-18 14:09:40.159285
FLWA_PBMC 2020-05-18 14:09:40.159285
453W_PBMC 2020-05-18 14:09:40.159285
684C_PBMC 2020-05-18 14:09:40.159285
CZJE_PBMC 2020-05-18 14:09:40.159285
DZQV_PBMC 2020-05-18 14:09:40.159285
925L_PBMC 2020-05-18 14:09:40.159285
9JD4_PBMC 2020-05-18 14:09:40.159285
G4YW_PBMC 2020-05-18 14:09:40.159285
4DUY_PBMC 2020-05-18 14:09:40.159285
36TS_PBMC 2020-05-18 14:09:40.159285
CR3L_PBMC 2020-05-18 14:09:40.159285
1 day, 22:01:07.546749


In [15]:
#real_prop = real_prop.drop('Progenitors',axis=1)

In [16]:
proportions_NuSVR_cibersort = proportions_NuSVR_cibersort.loc[real_prop.index,real_prop.columns]

In [17]:
clusters = real_prop.columns
for cluster in clusters:
    s,i,r,p,e = sci.stats.linregress(real_prop.loc[:,cluster],proportions_NuSVR_cibersort.loc[:,cluster]) 
    print(cluster,r,p)

B Naive 0.4735792290967454 0.11990114090416121
Plasmablasts 0.6035663817422183 0.037710003703494595
T CD4 Naive 0.709195849447193 0.009802021580580397
T CD8 Naive 0.494585141018253 0.10213202647550029
T gd Vd2 0.752881290034842 0.004709872299206708
T gd non-Vd2 0.9281076971412516 1.3393717077469222e-05
MAIT 0.6084967542183359 0.035765287967589085
NK 0.8283179916125382 0.0008737445726808219
pDCs 0.6393935899274007 0.02517413460703693
mDCs 0.7473934508259618 0.005203799268494771
Monocytes C 0.1385312612487221 0.6676572658065936
Neutrophils LD 0.29638512062642963 0.34956802615647453
Basophils LD 0.8425100320857674 0.0005820739516582865
B Memory 0.0 1.0
T CD4 Memory 0.21041541479827394 0.5115565607962059
T CD8 Memory 0.9517725267247025 1.8944833159800734e-06
Monocytes NC+I 0.6019387547085076 0.03836821791829773


## ImmunoState

In [11]:
immunoStates = pd.read_csv('./immunoStates.csv',index_col=0,delimiter=';')
immunoStates

,CD14_positive_monocyte,CD16_positive_monocyte,CD4_positive_alpha_beta_T_cell,CD56bright_natural_killer_cell,CD56dim_natural_killer_cell,CD8_positive_alpha_beta_T_cell,MAST_cell,basophil,eosinophil,gamma_delta_T_cell,hematopoietic_progenitor,macrophage_m0,macrophage_m1,macrophage_m2,memory_B_cell,myeloid_dendritic_cell,naive_B_cell,neutrophil,plasma_cell,plasmacytoid_dendritic_cell
Gene,,,,,,,,,,,,,,,,,,,,
ABCA5,"18,35739316","10,10510825","43,08169689","63,34601174","73,07646838","48,07726769","29,59769478","25,89600813","79,50603251","39,50528962","26,24504449","25,20067931","24,63089669","24,96150249","32,71537791","25,80346609","27,65254384","26,22902496","38,17628543","39,97198466"
ABCB1,"13,46795258","10,76767912","34,89057521","145,813992","119,9083196","57,4775539","17,73919118","20,12920591","9,894062673","64,56254989","48,18348373","25,32911022","15,4641323","13,56698818","22,47143424","16,53728547","72,40094318","23,32784129","14,96364506","24,63094656"
ABHD5,"62,87856759","62,18461702","42,79990627","39,96361029","72,5112559","32,9409149","36,50364596","305,9038895","45,14522841","37,270202","29,43526135","42,27956334","108,2592294","85,51079003","31,07959107","48,10861369","24,3727958","160,3476733","20,91145325","43,05606653"
ACSM3,"16,67459145","20,60276156","28,55981184","22,37620719","15,92140434","26,57111106","141,2194725","29,87160081","32,40856344","16,13130007","64,50549273","19,4008541","17,93519339","17,67975881","21,2418919","16,49207962","22,07819592","26,25922365","23,35575398","25,03972973"
ADAM19,"59,01350143","25,44391341","75,92887949","27,9261372","24,42580812","40,25628181","17,35817639","101,6320192","238,4446642","54,9048683","25,08670694","27,3222873","36,51258648","33,11254006","101,2328466","71,16534465","144,9819635","83,5209787","78,25572265","50,67043957"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZNF180,"42,68408653","29,38606808","37,49038396","28,4626332","48,37618697","26,0460025","42,5096755","13,89932646","38,80974986","54,28533277","41,74896977","44,66976239","11,15989935","10,36369041","23,67205929","21,24964712","21,9479547","36,94482119","15,48706342","15,10172004"
ZNF189,"67,02029489","32,68412391","48,39242603","49,64192421","44,14307868","41,59917211","67,81420799","64,52754548","48,31971202","103,4665381","145,8562104","52,92624409","51,87521317","47,87459943","52,43125502","65,04194247","50,61358264","71,16705874","49,14003472","38,54822451"
ZNF34,"37,69218873","86,51823718","35,83927608","42,73919984","35,5281306","34,69316866","26,27288294","86,37070099","12,85479507","36,12762412","34,85830678","57,51614782","42,12720188","48,46592788","21,93093196","46,60448229","23,09076621","33,73785521","25,29185512","20,96701087"


In [12]:
sorted_immunoStates = sorted_bulks_paper.loc[np.intersect1d(immunoStates.index,sorted_bulks_paper.index),:]

In [13]:
sorted_immunoStates.shape

(268, 17)

In [14]:
samples_immunoStates = samples.loc[np.intersect1d(samples.index,sorted_immunoStates.index),:]
samples_immunoStates.shape

(268, 13)

In [15]:
import datetime
time0 = datetime.datetime.now()
from sklearn.svm import NuSVR
proportions_NuSVR_immunoStates = pd.DataFrame(columns=sorted_immunoStates.columns)
regr_NuSVR = NuSVR(nu=0.5,C=0.5,kernel='linear') 
for column in samples_immunoStates.columns:
    regr_NuSVR.fit(sorted_immunoStates, samples_immunoStates.loc[:,column])
    proportions_NuSVR_immunoStates.loc[column] = regr_NuSVR.coef_[0]
    
    proportions_NuSVR_immunoStates.index = [x[:4] for x in proportions_NuSVR_immunoStates.index]
    proportions_NuSVR_immunoStates = normalize_proportions(proportions_NuSVR_immunoStates, copy = False)
time1 = datetime.datetime.now()
print(time1-time0)

0:10:24.239381


In [16]:
proportions_NuSVR_immunoStates = proportions_NuSVR_immunoStates.loc[real_prop.index,real_prop.columns]

In [51]:
clusters = real_prop.columns
sum_r = 0
sum_p = 0
for cluster in clusters:
    s,i,r,p,e = sci.stats.linregress(real_prop.loc[:,cluster],proportions_NuSVR_immunoStates.loc[:,cluster]) 
    sum_r+=r
    sum_p+=p
    print(round(r,2))
print(sum_r/17)
sum_p/17

0.92
0.66
0.28
0.5
-0.19
0.56
0.47
0.4
0.6
0.4
0.42
0.7
0.85
0.33
0.18
0.74
0.68
0.4993293154066027


0.16155129502538507

## MPC

In [37]:
MPC = pd.read_csv('./MCP-counter.csv',index_col=0,delimiter=';')
MPC

['CHRM3-AS2',
 'THEMIS',
 'THEMIS',
 'CD6',
 'MAL',
 'CD5',
 'CD28',
 'CD3G',
 'ICOS',
 'FLT3LG',
 'TNFRSF25',
 'TNFRSF25',
 'TNFRSF25',
 'CD28',
 'CD3D',
 'CD6',
 'AV8S2',
 'TRAT1',
 'TNFRSF25',
 'SIRPG',
 'CD5',
 'PBX4',
 'CTLA4',
 'MGC40069',
 'CD8B',
 'CD8B',
 'CD8B',
 'CD8B',
 'KLRC4-KLRK1',
 'KLRC4-KLRK1',
 'CD8A',
 'KLRC4-KLRK1',
 'KLRC4-KLRK1',
 'KLRC1',
 'KLRC1',
 'KLRC3',
 'KLRD1',
 'KLRD1',
 'KLRD1',
 'KLRC4',
 'FGFBP2',
 'EOMES',
 'GNLY',
 'FCRL3',
 'STAP1',
 'SNX29P1',
 'SNX29P1',
 'SNX29P1',
 'CD79A',
 'CLLU1',
 'CLLU1',
 'BANK1',
 'LOC100129447',
 'FCRL2',
 nan,
 nan,
 'ABCB4',
 'CD22',
 'CD79A',
 'CD79B',
 'CR2',
 'CD19',
 'ABCB4',
 'MS4A1',
 'IGHV5-78',
 'IGHM',
 'IGHA1',
 'IGH',
 'IGHA1',
 'IGHA2',
 'IGK',
 'IGK',
 'IGKV1-17',
 'IGHG3',
 'IGLJ3',
 ' IGHD',
 'IGLJ3',
 'IGK',
 'IGHD',
 'IGKC',
 'IGKV2D-28',
 'IGKC',
 'IGHA1',
 'IGHD',
 'AL928768',
 'IGK',
 'IGLC1',
 'LOC101929272',
 'IGHD',
 'IGLV4-60',
 'IGKC',
 'IGKV1-37',
 'IGHM',
 'IGHA1',
 'IGKC',
 'IGHG1',
 'IGHA1

In [38]:
sorted_MPC = sorted_bulks_paper.loc[np.intersect1d(list(MPC.loc[:,'Symbol']),sorted_bulks_paper.index),:]

In [39]:
sorted_MPC.shape

(228, 17)

In [40]:
samples_MPC = samples.loc[np.intersect1d(samples.index,sorted_MPC.index),:]
samples_MPC.shape

(228, 13)

In [41]:
import datetime
time0 = datetime.datetime.now()
from sklearn.svm import NuSVR
proportions_NuSVR_MPC = pd.DataFrame(columns=sorted_MPC.columns)
regr_NuSVR = NuSVR(nu=0.5,C=0.5,kernel='linear') 
for column in samples_MPC.columns:
    regr_NuSVR.fit(sorted_MPC, samples_MPC.loc[:,column])
    proportions_NuSVR_MPC.loc[column] = regr_NuSVR.coef_[0]
    
    proportions_NuSVR_MPC.index = [x[:4] for x in proportions_NuSVR_MPC.index]
    proportions_NuSVR_MPC = normalize_proportions(proportions_NuSVR_MPC, copy = False)
time1 = datetime.datetime.now()
print(time1-time0)

0:53:45.632395


In [42]:
proportions_NuSVR_MPC = proportions_NuSVR_MPC.loc[real_prop.index,real_prop.columns]

In [48]:
clusters = real_prop.columns
sum_r = 0
sum_p = 0
for cluster in clusters:
    s,i,r,p,e = sci.stats.linregress(real_prop.loc[:,cluster],proportions_NuSVR_MPC.loc[:,cluster]) 
    sum_r+=r
    sum_p+=p
    print(round(r,2))
print(sum_r/17)
sum_p/17

0.78
0.73
0.49
0.43
0.06
-0.24
-0.08
0.44
-0.29
0.04
0.26
0.62
0.0
0.09
0.44
-0.07
-0.18
0.2075230932581721


0.445974097801319

In [50]:
64*60


3840